In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [157]:
!pip install PyPDF2

import PyPDF2
import torch
from transformers import AutoTokenizer, T5EncoderModel

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = T5EncoderModel.from_pretrained("google-t5/t5-small")

In [189]:
input_ids = tokenizer('India is in Asia', return_tensors = 'pt').input_ids
query_ids = tokenizer('Where is the India? ', return_tensors = 'pt').input_ids

In [190]:
outputs = model(input_ids=input_ids)
query = model(input_ids = query_ids)

In [191]:
last_hidden_states = outputs.last_hidden_state
last_hidden_states.shape

torch.Size([1, 5, 512])

In [192]:
last_hidden_states_query = query.last_hidden_state
last_hidden_states_query.shape

torch.Size([1, 6, 512])

In [193]:
# Apply max pooling along the token dimension (dim=1)
pooled_last_hidden_states = torch.max(last_hidden_states, dim=1).values.unsqueeze(0)  # Add batch dimension
pooled_last_hidden_states_query = torch.max(last_hidden_states_query, dim=1).values.unsqueeze(0)  # Add batch dimension

# Reshape pooled_last_hidden_states_query to match the shape of pooled_last_hidden_states
pooled_last_hidden_states_query = pooled_last_hidden_states_query.view(1, -1)

# Compute dot product between the pooled representations
similarity = torch.matmul(pooled_last_hidden_states, pooled_last_hidden_states_query.T)

# Print similarity score
print("Similarity Score:", similarity.item())


Similarity Score: 15.463321685791016
